# SDN-DNN with NSL-KDD

This notebook applies the DNN proposed by Tang et al. in [10.1109/WINCOM.2016.7777224](https://doi.org/10.1109/WINCOM.2016.7777224) on the NSL-KDD dataset.  
The NSL-KDD version used is the [preprocessed one by the University of New Brunswick, Canada](http://www.unb.ca/cic/datasets/nsl.html).

## Data loading and prep

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

header_col = pd.read_csv(os.path.join('NSL_KDD', 'Field Names.csv'), header=None)
header_col = header_col.append(pd.DataFrame([['label','symbolic'],['unknown','continuous']]))

header_names = header_col[0].values

### Training Set

In [2]:
ftrain = os.path.join('NSL_KDD','KDDTrain+.csv')
kdd_train = pd.read_csv(ftrain, header=None, names=header_names)

# split off labels
kdd_train_labels = kdd_train['label']

# only keep columns that are actually used
used_fields = ['duration', 'protocol_type', 'src_bytes', 'dst_bytes', 'count', 'srv_count']
kdd_train = kdd_train.filter(used_fields)
kdd_train.head()

,duration,protocol_type,src_bytes,dst_bytes,count,srv_count
0,0,tcp,491,0,2,2
1,0,udp,146,0,13,1
2,0,tcp,0,0,123,6
3,0,tcp,232,8153,5,5
4,0,tcp,199,420,30,32


### Test Set

In [3]:
ftest = os.path.join('NSL_KDD','KDDTest+.csv')
kdd_test = pd.read_csv(ftest, header=None, names=header_names)

# split off labels
kdd_test_labels = kdd_test['label']

# only keep columns that are actually used
kdd_test = kdd_test.filter(used_fields)

kdd_test.tail()

,duration,protocol_type,src_bytes,dst_bytes,count,srv_count
22538,0,tcp,794,333,1,1
22539,0,tcp,317,938,2,11
22540,0,tcp,54540,8314,5,10
22541,0,udp,42,42,4,6
22542,0,tcp,0,0,4,10


Encode the lables to a numeric list

In [4]:
from keras.preprocessing.text import Tokenizer
# tokenize the LABELS
label_tokenizer = Tokenizer(num_words=50, filters='')
label_tokenizer.fit_on_texts((kdd_train_labels.append(kdd_test_labels)).values)

# Run the fitted tokenizer on the label column and save the encoded data as dataframe
kdd_train_labels = label_tokenizer.texts_to_sequences(kdd_train_labels)
kdd_train_labels = np.concatenate(kdd_train_labels).ravel()

# as the Encoder documentation states, 0 will never assigned to a label.
# I, on the other hand, need an index starting with 0. So we substract 1 of all classes.
kdd_train_labels = kdd_train_labels -1

# Do the same for the test labels
kdd_test_labels = label_tokenizer.texts_to_sequences(kdd_test_labels)
kdd_test_labels = np.concatenate(kdd_test_labels).ravel()
kdd_test_labels = kdd_test_labels -1

Using TensorFlow backend.


In [5]:
def f(x):
    return 1 if x > 0 else 0
f = np.vectorize(f)

# We only want to know if it's benign or not, so we switch to 0 or 1
kdd_train_labels = f(kdd_train_labels)
kdd_test_labels = f(kdd_test_labels)

In [6]:
# build a big dataframe out of both sets to train the tokenizer
full = pd.concat([kdd_train, kdd_test])
kdd_test.tail()

,duration,protocol_type,src_bytes,dst_bytes,count,srv_count
22538,0,tcp,794,333,1,1
22539,0,tcp,317,938,2,11
22540,0,tcp,54540,8314,5,10
22541,0,udp,42,42,4,6
22542,0,tcp,0,0,4,10


In [7]:
# tokenize the protocol_type column
protocol_tokenizer = Tokenizer(num_words=50, filters='')
protocol_tokenizer.fit_on_texts(full['protocol_type'])

train_enc = protocol_tokenizer.texts_to_sequences(kdd_train['protocol_type'])
test_enc = protocol_tokenizer.texts_to_sequences(kdd_test['protocol_type'])

In [8]:
kdd_train.drop('protocol_type', axis=1, inplace=True) # drop named column
kdd_train = pd.concat([kdd_train, pd.DataFrame(train_enc,columns=['protocol_type'])], axis=1, sort=False)
kdd_train.tail()

,duration,src_bytes,dst_bytes,count,srv_count,protocol_type
125968,0,0,0,184,25,1
125969,8,105,145,2,2,2
125970,0,2231,384,1,1,1
125971,0,0,0,144,8,1
125972,0,151,0,1,1,1


In [9]:
kdd_test.drop('protocol_type', axis=1, inplace=True)
kdd_test = pd.concat([kdd_test, pd.DataFrame(test_enc,columns=['protocol_type'])], axis=1, sort=False)
kdd_test.tail()

,duration,src_bytes,dst_bytes,count,srv_count,protocol_type
22538,0,794,333,1,1,1
22539,0,317,938,2,11,1
22540,0,54540,8314,5,10,1
22541,0,42,42,4,6,2
22542,0,0,0,4,10,1


## Data Normalization

In [10]:
# Fit on train data, transform test data with that
# https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
kdd_train = min_max_scaler.fit_transform(kdd_train)
kdd_test = min_max_scaler.transform(kdd_test)

In [11]:
print("Shape of the final netflow dataset:", kdd_train.shape)
print("Outer type:", type(kdd_train))
print("Single entry type:", type(kdd_train[0]))

Shape of the final netflow dataset: (125973, 6)
Outer type: <class 'numpy.ndarray'>
Single entry type: <class 'numpy.ndarray'>


## Runtime Preqs

In [12]:
from datetime import datetime
from os.path import exists, join

# Define some semi-global stuff

batch_size = 10
epochs = 100
learn_rate = 0.001

run_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
runtype_name = 'nsl-kdd-sdn-dnn'
log_folder_path = os.path.join('logs',runtype_name + '-{}'.format(run_date))

print('Using set globals: ')
print('Batch Size:', batch_size)
print('Epochs:', epochs)
print('Learn rate:', learn_rate)

Using set globals: 
Batch Size: 10
Epochs: 100
Learn rate: 0.001


In [13]:
print("No of scaled train entries:\t", len(kdd_train))
print("No of train labels:\t\t", len(kdd_train_labels))
print("-----------")
print("No of test entries:\t\t", len(kdd_test))
print("No of test labels:\t\t", len(kdd_test_labels))

No of scaled train entries:	 125973
No of train labels:		 125973
-----------
No of test entries:		 22543
No of test labels:		 22543


## Building and Training the Model

In [14]:
# Time for some nice vizualization stuff. Set this up and include as callback, then:
# tensorboard --logdir=path/to/logdir
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard

callbacks = [
#     EarlyStopping(    # No early stopping! We want the full 100 epochs
#         monitor='acc', # Which metric to monitor
#         patience=3     # Interrupt training after acc has stopped improving for more than 1 epoch
#     ),
    ModelCheckpoint(
        filepath='models/'+runtype_name+'-{}.h5'.format(run_date),
        monitor='val_loss',   
        save_best_only=True    # Only save one. Only overwrite this one if val_loss has improved
    ),
    TensorBoard(
        log_dir=log_folder_path,
        #histogram_freq=1,     # Record activation histograms every epoch
        #embeddings_freq=1,     # Record embedding data every epoch -> There's something wrong with the embeddings here. Keras crashed with them enabled
        #embeddings_layer_names=['LSTMnet'],
        #embeddings_metadata='metadata.tsv',
        #embeddings_data=data_test,
       # batch_size=batch_size
    )
]

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from keras.utils import plot_model

# see implementation/sdn-dnn.py for details, alternatives and comments
# TODO: Isn't there some kind of dropout missing?

model = Sequential()
model.add(Dense(12, activation='relu', input_dim=6))
model.add(Dense(6, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(RMSprop(lr=learn_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()
# plot_model(model, to_file='model-sdn-dnn-alt.png', show_layer_names=True, show_shapes=True)

history = model.fit(kdd_train, kdd_train_labels, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    verbose=1,
                    validation_data=(kdd_test, kdd_test_labels),
                    callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                84        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 78        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 21        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 8         
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________
Train on 125973 samples, validate on 22543 samples
Epoch 1/100
125973/125973 [==============================] - 11s 90us/step - loss: 0.2558 - acc: 0.9081 - val_loss: 0.7071 - val_acc: 0.7031
Epoch 2/100
125973/125973 [==============================] - 11s 88us/step - loss: 0

KeyboardInterrupt: 